## <h1><span style="color: red;">Bireylerin Sigara İçip İçmeme Durumlarını Tahmin Etme</span></h1>

* 10 Kümeye Ayırma ve Eğitim/Test Setlerini Oluşturma

In [1]:
#Kullanacağımız kütüphanelerimizi yüklüyoruz
from sklearn import tree
import pandas as pd
import os
from sklearn import preprocessing
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

print(os.getcwd())

C:\Users\musta\Desktop\proje


### VERİ SETİNİ YÜKLE VE İŞLE

In [2]:
df=pd.read_csv("Cardio_train.csv" ,delimiter=";")

# Vücut Kitle İndeksi (BMI) ekleyin
df['BMI'] = df['weight'] / (df['height']/100) ** 2

# Girdi (features) ve Çıktı (labels) değişkenlerini belirleyin
X = df.drop(columns=['id', 'cardio', 'smoke'])
y_cardio = df['cardio']
y_smoke = df['smoke']

In [3]:
df

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,BMI
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0,21.967120
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1,34.927679
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1,23.507805
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1,28.710479
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0,23.011177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,99993,19240,2,168,76.0,120,80,1,1,1,0,1,0,26.927438
69996,99995,22601,1,158,126.0,140,90,2,2,0,0,1,1,50.472681
69997,99996,19066,2,183,105.0,180,90,3,1,0,1,0,1,31.353579
69998,99998,22431,1,163,72.0,135,80,1,2,0,0,0,1,27.099251


In [4]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)
splits = list(kf.split(X))

# İlk split'i kullanarak eğitim ve test setlerini oluşturun
train_index, test_index = splits[0]
X_train, X_test = X.iloc[train_index], X.iloc[test_index]
y_cardio_train, y_cardio_test = y_cardio.iloc[train_index], y_cardio.iloc[test_index]
y_smoke_train, y_smoke_test = y_smoke.iloc[train_index], y_smoke.iloc[test_index]


In [5]:
# İlk split'i kullanarak eğitim ve test setlerini oluşturun
train_index, test_index = splits[0]
X_train, X_test = X.iloc[train_index], X.iloc[test_index]
y_smoke_train, y_smoke_test = y_smoke.iloc[train_index], y_smoke.iloc[test_index]

### RandomForest Modelini Eğit ve Test Et


In [6]:
# RandomForest modeli oluşturun
rf_smoke = RandomForestClassifier(random_state=42)

# Modeli eğitin
rf_smoke.fit(X_train, y_smoke_train)

# Tahmin yapın
y_smoke_pred = rf_smoke.predict(X_test)

# Performans değerlendirmesi yapın
print("Smoke Prediction Accuracy:", accuracy_score(y_smoke_test, y_smoke_pred))
print("Smoke Confusion Matrix:\n", confusion_matrix(y_smoke_test, y_smoke_pred))
print("Smoke Classification Report:\n", classification_report(y_smoke_test, y_smoke_pred))

Smoke Prediction Accuracy: 0.9194285714285715
Smoke Confusion Matrix:
 [[6299   92]
 [ 472  137]]
Smoke Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.99      0.96      6391
           1       0.60      0.22      0.33       609

    accuracy                           0.92      7000
   macro avg       0.76      0.61      0.64      7000
weighted avg       0.90      0.92      0.90      7000



### Çapraz Doğrulama 

In [7]:
# 10 katlı çapraz doğrulama yapın
cv_scores_smoke = cross_val_score(rf_smoke, X, y_smoke, cv=kf)

print("Smoke Cross-Validation Scores:", cv_scores_smoke)
print("Smoke CV Mean Score:", np.mean(cv_scores_smoke))

Smoke Cross-Validation Scores: [0.91942857 0.917      0.91857143 0.91557143 0.91828571 0.91714286
 0.91957143 0.91342857 0.916      0.91428571]
Smoke CV Mean Score: 0.9169285714285715


### %90 Eğitim, %10 Test


In [8]:
# Veriyi %90 Eğitim, %10 Test olarak ayırın
X_train, X_test, y_smoke_train, y_smoke_test = train_test_split(X, y_smoke, test_size=0.1, random_state=42)

# Modeli yeniden eğitin ve test edin
rf_smoke.fit(X_train, y_smoke_train)
y_smoke_pred = rf_smoke.predict(X_test)

print("Smoke Prediction Accuracy:", accuracy_score(y_smoke_test, y_smoke_pred))
print("Smoke Confusion Matrix:\n", confusion_matrix(y_smoke_test, y_smoke_pred))
print("Smoke Classification Report:\n", classification_report(y_smoke_test, y_smoke_pred))

Smoke Prediction Accuracy: 0.9195714285714286
Smoke Confusion Matrix:
 [[6300   91]
 [ 472  137]]
Smoke Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.99      0.96      6391
           1       0.60      0.22      0.33       609

    accuracy                           0.92      7000
   macro avg       0.77      0.61      0.64      7000
weighted avg       0.90      0.92      0.90      7000



In [9]:
#Sigara içen ve içmeyenlerin toplam sayısını öğrenelim 
df['smoke'].value_counts()

smoke
0    63831
1     6169
Name: count, dtype: int64

## Normalize Hali

In [14]:
# Sigara içen ve içmeyenleri ayır
smokers = df[df['smoke'] == 1]
nonsmokers = df[df['smoke'] == 0]

# Sigara içmeyenlerden, sigara içenlerin sayısına eşit sayıda rastgele örnek seç
nonsmokers_downsampled = nonsmokers.sample(n=len(smokers), random_state=42)

# Dengelenmiş veri setini oluştur
balanced_df = pd.concat([smokers, nonsmokers_downsampled])

# Veriyi karıştır ve indeksleri sıfırla
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Girdi (X) ve Çıktı (y) değişkenlerini ayır
X = balanced_df.drop('smoke', axis=1)  # 'smoke' sütunu çıkarılıyor
y = balanced_df['smoke']

In [15]:
# Veriyi %90 eğitim, %10 test olarak ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# RandomForest modelini oluştur
rf_classifier = RandomForestClassifier(random_state=42)

# Modeli eğit
rf_classifier.fit(X_train, y_train)

# Test seti üzerinde tahmin yap
y_pred = rf_classifier.predict(X_test)

# Performans metriklerini hesapla
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

In [17]:
# Performans metriklerini yazdır
print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

Accuracy: 0.7860615883306321
Confusion Matrix:
 [[434 165]
 [ 99 536]]
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.72      0.77       599
           1       0.76      0.84      0.80       635

    accuracy                           0.79      1234
   macro avg       0.79      0.78      0.78      1234
weighted avg       0.79      0.79      0.79      1234



In [16]:
from sklearn.model_selection import GridSearchCV

# RandomForest parametre grid'i oluştur
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# GridSearchCV objesi oluştur
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42), 
                           param_grid=param_grid, 
                           cv=5, 
                           n_jobs=-1, 
                           verbose=2)

# Grid search ile modeli eğit
grid_search.fit(X_train, y_train)

# En iyi parametreleri ve modeli al
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# En iyi modelle tahmin yap
y_pred = best_model.predict(X_test)

# Performans metriklerini yazdır
print("Best Params:", best_params)
print("Accuracy:", accuracy_score(y_test, y_pred))

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Params: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}
Accuracy: 0.7982171799027553
